<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/home/ga_aiot/anaconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).

In [3]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [4]:
datadays = 1825


In [5]:
#Download BTC Data 
import requests

url = "https://min-api.cryptocompare.com/data/v2/histoday"
params = {
    "fsym": "BTC",
    "tsym": "USD",
    "limit": datadays,  # 5 years = 1825 days
    "aggregate": 1
}
headers = {
    "authorization": "Apikey 16c99daa7cc2352c372d2186252ba1afeabbdd723f13b4fcdf909f681f0318f3"
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['Data']['Data'])
    df['date'] = pd.to_datetime(df['time'], unit='s').dt.date
    df['tic'] = 'btc'
    df.drop(['conversionType', 'conversionSymbol'], axis=1, inplace=True)
    print(df.head())

else:
    print("Error:", response.status_code)



         time      high       low      open  volumefrom      volumeto  \
0  1519516800   9873.79   9329.44   9705.73    82430.45  7.930932e+08   
1  1519603200  10457.51   9411.82   9610.11   117416.47  1.181729e+09   
2  1519689600  10879.38  10154.24  10326.50    99772.39  1.060841e+09   
3  1519776000  11067.76  10303.14  10594.76   105434.79  1.126075e+09   
4  1519862400  11090.30  10247.56  10334.44    88432.44  9.524721e+08   

      close        date  tic  
0   9610.11  2018-02-25  btc  
1  10326.50  2018-02-26  btc  
2  10594.76  2018-02-27  btc  
3  10334.44  2018-02-28  btc  
4  10929.37  2018-03-01  btc  


In [6]:
fwd_testperiod = 300 

TRAIN_START_DATE = df.date.sort_values(ascending=False)[0]
TRAIN_END_DATE = df.date.sort_values(ascending=False)[datadays-fwd_testperiod]
TRADE_START_DATE = TRAIN_END_DATE
TRADE_END_DATE = df.date.sort_values(ascending=True)[datadays-1]

print(TRAIN_START_DATE,TRAIN_END_DATE,TRADE_START_DATE,TRADE_END_DATE)


2018-02-25 2022-04-30 2022-04-30 2023-02-23


In [7]:
df.shape

(1826, 9)

In [8]:
df.sort_values(['date','tic'],ignore_index=True).head()

,time,high,low,open,volumefrom,volumeto,close,date,tic
0,1519516800,9873.79,9329.44,9705.73,82430.45,7.930932e+08,9610.11,2018-02-25,btc
1,1519603200,10457.51,9411.82,9610.11,117416.47,1.181729e+09,10326.50,2018-02-26,btc
2,1519689600,10879.38,10154.24,10326.50,99772.39,1.060841e+09,10594.76,2018-02-27,btc
3,1519776000,11067.76,10303.14,10594.76,105434.79,1.126075e+09,10334.44,2018-02-28,btc
4,1519862400,11090.30,10247.56,10334.44,88432.44,9.524721e+08,10929.37,2018-03-01,btc


In [9]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [10]:
list_ticker = processed["tic"].tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = processed
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [11]:
processed_full.sort_values(['date'],ignore_index=True).head(10)

,time,high,low,open,volumefrom,volumeto,close,date,tic,macd,boll_ub,boll_lb,rsi_7,rsi_14,cci_14,dx_30,close_7_sma,close_14_sma
0,1519516800,9873.79,9329.44,9705.73,82430.45,7.930932e+08,9610.11,2018-02-25,btc,0.000000,10981.433454,8955.176546,100.000000,100.000000,66.666667,100.000000,9610.110000,9610.110000
1,1519603200,10457.51,9411.82,9610.11,117416.47,1.181729e+09,10326.50,2018-02-26,btc,16.072853,10981.433454,8955.176546,100.000000,100.000000,66.666667,100.000000,9968.305000,9968.305000
2,1519689600,10879.38,10154.24,10326.50,99772.39,1.060841e+09,10594.76,2018-02-27,btc,28.798731,11195.197883,9159.048784,100.000000,100.000000,100.000000,100.000000,10177.123333,10177.123333
3,1519776000,11067.76,10303.14,10594.76,105434.79,1.126075e+09,10334.44,2018-02-28,btc,24.482172,11062.462188,9370.442812,74.392684,76.904013,69.039476,100.000000,10216.452500,10216.452500
4,1519862400,11090.30,10247.56,10334.44,88432.44,9.524721e+08,10929.37,2018-03-01,btc,45.294841,11330.324002,9387.747998,84.782566,85.274486,79.086649,89.696643,10359.036000,10359.036000
5,1519948800,11191.94,10801.45,10929.37,77195.49,8.529085e+08,11043.12,2018-03-02,btc,61.818701,11505.862142,9440.237858,86.045533,86.297066,99.560321,90.639326,10473.050000,10473.050000
6,1520035200,11530.66,11043.12,11043.12,71279.36,8.122001e+08,11465.36,2018-03-03,btc,90.478992,11819.628568,9409.988574,89.735005,89.274472,126.512523,92.940305,10614.808571,10614.808571
7,1520121600,11539.79,11084.01,11465.36,61016.39,6.907273e+08,11504.42,2018-03-04,btc,110.099501,12006.605681,9445.414319,90.019786,89.501720,103.111941,92.989955,10885.424286,10726.010000
8,1520208000,11694.15,11431.55,11503.94,68323.51,7.914719e+08,11440.73,2018-03-05,btc,118.851231,12094.596644,9516.250023,85.507054,86.291226,102.840508,93.780485,11044.600000,10805.423333
9,1520294400,11441.65,10589.28,11440.33,109876.55,1.210081e+09,10735.45,2018-03-06,btc,87.001813,12014.675662,9582.176338,51.896453,60.437547,21.622952,16.891866,11064.698571,10798.426000


In [12]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

1525
299


In [13]:
train.tail()

,time,high,low,open,volumefrom,volumeto,close,date,tic,macd,boll_ub,boll_lb,rsi_7,rsi_14,cci_14,dx_30,close_7_sma,close_14_sma
1520,1650844800,40590.80,38234.70,39466.26,38636.23,1.514244e+09,40437.52,2022-04-25,btc,-825.307048,43460.418892,38385.287108,47.981621,45.215137,-77.428653,34.694936,40348.078571,40362.900000
1521,1650931200,40797.85,37726.59,40437.52,38188.24,1.492484e+09,38117.39,2022-04-26,btc,-955.792582,43269.102330,38070.564670,30.159583,36.587092,-158.537462,38.140415,39864.214286,40222.237857
1522,1651017600,39465.62,37900.60,38117.39,34510.32,1.341341e+09,39252.04,2022-04-27,btc,-956.619225,42771.307750,38146.942250,42.372250,42.378054,-132.539032,38.140415,39560.884286,40086.645714
1523,1651104000,40386.50,38892.10,39252.04,29981.12,1.188928e+09,39749.75,2022-04-28,btc,-906.661874,42499.077789,38166.809211,47.105660,44.761020,-41.065911,24.533447,39454.995714,40072.225714
1524,1651190400,39922.60,38179.15,39749.75,24745.92,9.640201e+08,38594.22,2022-04-29,btc,-949.368201,42098.109608,38150.338392,38.532817,40.566459,-106.807869,30.221722,39294.735714,39931.549286


In [14]:
trade.head()

,time,high,low,open,volumefrom,volumeto,close,date,tic,macd,boll_ub,boll_lb,rsi_7,rsi_14,cci_14,dx_30,close_7_sma,close_14_sma
0,1651276800,38788.01,37607.77,38594.22,13696.14,5.252913e+08,37650.13,2022-04-30,btc,-1047.320587,41924.927712,37873.156288,32.836573,37.476436,-160.418069,34.454254,39038.187143,39735.662143
1,1651363200,38668.00,37461.76,37650.13,17299.60,6.590303e+08,38480.53,2022-05-01,btc,-1045.885896,41964.828730,37727.637270,41.683103,41.684092,-119.177879,35.522592,38897.368571,39649.402857
2,1651449600,39155.96,38057.31,38480.53,29810.45,1.152558e+09,38513.01,2022-05-02,btc,-1030.251926,41964.006357,37571.094643,42.031527,41.848934,-76.798831,28.065626,38622.438571,39485.258571
3,1651536000,38644.50,37519.23,38513.01,28729.95,1.095654e+09,37725.38,2022-05-03,btc,-1069.093210,41871.471202,37321.134798,35.954163,38.972036,-108.810693,32.447105,38566.437143,39215.325714
4,1651622400,40004.37,37665.80,37725.38,36519.07,1.426470e+09,39680.21,2022-05-04,btc,-931.400162,41852.206960,37313.257040,54.855068,48.444935,-2.901529,13.732409,38627.604286,39094.244286


In [15]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_7',
 'rsi_14',
 'cci_14',
 'dx_30',
 'close_7_sma',
 'close_14_sma']

In [16]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 12


In [17]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 30,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e2
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


## Environment for Training



In [18]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [41]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [42]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {
    "n_steps": 8,
    "ent_coef": 0.01,
    "learning_rate": 0.001,
    "device":"cpu"
}

model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 8, 'ent_coef': 0.01, 'learning_rate': 0.001, 'device': 'cpu'}
Using cpu device


In [43]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=10000000) if if_using_a2c else None

### Agent 2: DDPG

In [44]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [45]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [46]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
    "device":"cpu"
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger 
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128, 'device': 'cpu'}
Using cpu device


In [47]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None
if trained_ppo is not None:
    trained_ppo.save("trained_ppo.zip")

### Agent 4: TD3

In [48]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cuda device


In [49]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [50]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 64,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results/sac


In [51]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

day: 1524, episode: 6560
begin_total_asset: 1000000.00
end_total_asset: 3628559.49
total_reward: 2628559.49
total_cost: 8943.66
total_trades: 1508
Sharpe: 0.661
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 76         |
|    time_elapsed    | 79         |
|    total_timesteps | 6100       |
| train/             |            |
|    actor_loss      | 1.36e+06   |
|    critic_loss     | 2.71e+13   |
|    ent_coef        | 0.182      |
|    ent_coef_loss   | 54.3       |
|    learning_rate   | 0.0001     |
|    n_updates       | 5999       |
|    reward          | -3235484.0 |
-----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 62         |
|    time_elapsed    | 195        |
|    total_timesteps | 12200      |
| train/             |            |
|    actor_loss      | 1.74e+06   |
|    critic

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [52]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [53]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()


In [54]:
trade.tail(50)

,time,high,low,open,volumefrom,volumeto,close,date,tic,macd,boll_ub,boll_lb,rsi_7,rsi_14,cci_14,dx_30,close_7_sma,close_14_sma
249,1672790400,16972.62,16651.02,16670.16,58841.99,9.906893e+08,16846.82,2023-01-04,btc,-100.485162,16979.102673,16453.901327,60.162364,50.546827,76.498688,6.576253,16651.850000,16713.090714
250,1672876800,16869.84,16764.64,16846.82,31174.48,5.245115e+08,16825.87,2023-01-05,btc,-84.312209,16990.340046,16459.406954,58.175896,49.987993,67.889683,6.576253,16679.654286,16713.795000
251,1672963200,17013.77,16687.42,16825.87,52488.70,8.836421e+08,16946.16,2023-01-06,btc,-61.084486,17015.714034,16450.593966,65.751117,53.188174,98.631423,0.998048,16729.181429,16725.594286
252,1673049600,16972.62,16905.39,16946.16,11628.45,1.969154e+08,16942.73,2023-01-07,btc,-42.463626,17040.889246,16445.417754,65.357318,53.083868,118.871273,0.998048,16787.955714,16733.135000
253,1673136000,17132.21,16913.28,16942.73,21159.60,3.591499e+08,17115.81,2023-01-08,btc,-13.583766,17083.597099,16470.011901,74.387902,57.602197,149.255897,9.176344,16859.661429,16753.614286
254,1673222400,17387.59,17103.20,17115.81,73779.59,1.271944e+09,17179.03,2023-01-09,btc,14.240897,17142.999926,16438.400074,76.948588,59.149740,184.386889,20.421369,16932.368571,16772.397143
255,1673308800,17484.36,17148.57,17179.03,57628.21,9.977649e+08,17442.44,2023-01-10,btc,56.891321,17279.237988,16364.293012,84.487602,64.898652,176.705530,24.221720,17042.694286,16825.390000
256,1673395200,17986.45,17324.71,17442.44,63571.70,1.113334e+09,17938.00,2023-01-11,btc,129.190413,17554.868984,16200.861016,90.969825,72.686491,213.533076,40.090037,17198.577143,16925.213571
257,1673481600,19092.31,17906.40,17938.00,122644.76,2.252898e+09,18849.00,2023-01-12,btc,257.035110,18090.010267,15872.522733,95.237799,81.022245,260.613696,59.873419,17487.595714,17083.625000
258,1673568000,19992.23,18717.45,18849.00,94611.39,1.820147e+09,19932.05,2023-01-13,btc,440.666310,18855.660590,15416.361410,97.123484,86.354153,250.833057,68.827814,17914.151429,17321.666429


In [56]:
#from stable_baselines3 import PPO
trained_model = trained_sac
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

hit end!


In [57]:
df_account_value.shape

(299, 2)

In [58]:
df_account_value.tail()

,date,account_value
294,2023-02-18,666377.80237
295,2023-02-19,656907.82237
296,2023-02-20,671772.94237
297,2023-02-21,661308.01237
298,2023-02-22,654191.35237


In [59]:
df_actions.tail()

,date,actions
293,2023-02-17,[0]
294,2023-02-18,[0]
295,2023-02-19,[0]
296,2023-02-20,[0]
297,2023-02-21,[0]


<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [60]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.300683
Cumulative returns    -0.345809
Annual volatility      0.514031
Sharpe ratio          -0.436996
Calmar ratio          -0.505906
Stability              0.345335
Max drawdown          -0.594345
Omega ratio            0.919901
Sortino ratio         -0.582168
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.987288
Daily value at risk   -0.065653
dtype: float64


In [61]:
df_account_value.loc[0,'date']

datetime.date(2022, 4, 30)

In [62]:
df_account_value.loc[len(df_account_value)-1,'date']

datetime.date(2023, 2, 22)

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value)